In [19]:
import xgboost as xgb
import pandas as pd
import numpy as np

In [20]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [4]:
from statistic import Statistic
from factors import Factor
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction
from decomposition_pca.encode_strategy import Encode

In [5]:
import category_encoders as ce
from category_encoders import HashingEncoder

In [6]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [7]:
# kyw3
#path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [8]:
str(list(db_teach.columns))

"['amount', 'amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'id', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status']"

In [9]:
df_extra_factor = pd.read_csv('/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/02_pure_data/kyw3_bayes_resolved.csv', dtype=str)
path_resolving_file = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/02_pure_data/kyw3_for_resolve.csv'
df_resolving = pd.read_csv(path_resolving_file, dtype=str, encoding='cp1251')

In [15]:
str(list(df_resolving))

"['order_id', 'type', 'status', 'date', 'processed', 'amount', 'currency', 'card_holder', 'email', 'phone', 'address', 'city_from_order', 'country', 'zip', 'ip', 'project_id', 'reported', 'status_chargeback', 'date_cb_only', 'card_masked', 'card_expire', 'amount_in_currency', 'date_only', 'id']"

In [10]:
df_resolving['bank_name'] = df_extra_factor.bank_name
df_resolving['region_name'] = df_extra_factor.region_name

In [11]:
col_names = ['bank_name', 'order_id', 'region_name', 'country', 'project_id', 'card_holder', 'address']
db_teach = db_teach.merge(df_resolving[col_names], left_on='order_id', right_on='order_id', how='left')
db_test = db_test.merge(df_resolving[col_names], left_on='order_id', right_on='order_id', how='left')

In [16]:
COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'phone_2_norm', 'longitude', 'latitude', 'is_gender_undefined', 'is_city_resolved']
cat_features = ['country', 'region_name']
COL_FACTORS = COL_FACTORS + cat_features

In [17]:
train_df = db_teach[COL_FACTORS].copy()
test_df = db_test[COL_FACTORS].copy()

## Encoders

In [ ]:
for col in COL_FACTORS:
    is_factor_encode = False
    if is_factor_encode:
        col_enc_name = 'enc_' + col
        print("col_enc_name={}" . format(col_enc_name))
        ret_val = Encode().ordered_target_st(db_teach, db_test, col)
        db_teach[col_enc_name] =  ret_val.get('encode_teach')
        db_test[col_enc_name] =  ret_val.get('test_encode_last')

In [22]:
encode_name = ""
if encode_name = 'CatBoostEncoder':
    encoder = ce.CatBoostEncoder(cols=cat_features)
    cat_train_df = train_df[cat_features]
    cat_test_df = test_df[cat_features]
    encoder.fit(cat_train_df, db_teach.status)
    trans_teach = encoder.transform(cat_train_df)
    trans_test = encoder.transform(cat_test_df)
    for cat_feature in cat_features:
        train_df[cat_feature] = trans_teach[cat_feature]
        test_df[cat_feature] = trans_test[cat_feature]
encoder.get_feature_names()    
data = {'num': trans_teach[cat_features[0]].values, 'country': cat_train_df[cat_features[0]].values}
df = (pd.DataFrame(data)).reset_index()
df.head(100)

In [33]:
encode_name = ""
import re
def get_two_digits(x:str)->str:    
    pattern = '^[0-9]+'
    if not isinstance(x, str):
        x = ''
    result = re.search(pattern, x)
    if result:
        s = result.group(0)
        retval = s[-2:]
    else:
        retval = '
    return retval
if encode_name = 'special_encode_country':
    cat_feature = 'country'
    train_df[cat_feature] = -trans_teach[cat_feature]
    test_df[cat_feature] = -trans_test[cat_feature]
    db_teach['help'] = db_teach.address.apply(lambda x: get_two_digits(x))
    db_test ['help'] = db_test.address.apply(lambda x: get_two_digits(x))
    train_df.country = np.where((db_teach.country == 'USA') & (db_teach.address.str.contains('^[0-9]')), db_teach.help.apply(pd.to_numeric), train_df.country)
    test_df.country = np.where((db_test.country == 'USA') & (db_test.address.str.contains('^[0-9]')), db_test.help.apply(pd.to_numeric), test_df.country)

In [28]:
encode_name = "hash"
if encode_name = 'hash_encode':
    cat_train_df = train_df[cat_features]
    cat_test_df = test_df[cat_features]
    hashing_enc = HashingEncoder(cols=cat_features, n_components=20).fit(cat_train_df, db_teach.status)
    X_train_hashing = hashing_enc.transform(cat_train_df.reset_index(drop=True))
    X_test_hashing = hashing_enc.transform(cat_test_df.reset_index(drop=True))
    col_hashing = list(X_train_hashing)
    for col in col_hashing:
        train_df[col] = X_train_hashing[col]
        test_df[col] = X_test_hashing[col]


In [37]:
str(list(train_df))

"['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'phone_2_norm', 'longitude', 'latitude', 'is_gender_undefined', 'is_city_resolved', 'country', 'region_name', 'col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19']"

In [38]:
train = train_df.apply(pd.to_numeric, errors="coerce")
test = test_df.apply(pd.to_numeric, errors="coerce")

In [39]:
replace_val = -9999
train = train.fillna(replace_val)
test = test.fillna(replace_val)
replace_val

-9999

In [40]:
train = train.values
test = test.values
label = db_teach.status

In [41]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [42]:
result_df_amount = None
weight = analyzer_prediction.get_xgb_weight()

In [43]:
# Kyw3
max_depths =[3]
nrounds = [80, 90]
etas = [0.2, 0.35]

In [44]:
for nround in nrounds:
    for eta in etas:
        for max_depth in max_depths:
            config = {'max_depth': max_depth, 'learning_rate': eta
                      , 'n_estimators': nround
                     }
            model = xgb.XGBClassifier(**config)
            
            model.fit(train, label, eval_metric = 'auc', sample_weight=weight)
             
            test_pred = model.predict_proba(test)
            db_test["probability"] = test_pred[:, 1]

            description = '-' . join([str(elem) for elem in (max_depth, nround, eta, 'encode country')])                    
            result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")

In [45]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]
col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [46]:
stat_best.iloc[:,:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
2,3-80-0.35-encode country,8.29,14.41,15.76,17.15,19.90,20.72,22.82,26.88,38.35,140.74
4,3-90-0.2-encode country,6.59,11.27,13.89,18.43,19.49,20.24,21.79,25.73,44.72,140.12
0,3-80-0.2-encode country,5.33,10.56,12.28,17.39,19.38,20.93,21.79,26.90,46.07,137.91
6,3-90-0.35-encode country,8.11,14.68,15.84,17.18,19.15,20.84,21.14,24.27,38.20,137.43


In [47]:
data = {'feature': train_df.columns , 'impotance': model.feature_importances_}
df = pd.DataFrame(data)
df.sort_values(by='impotance', ascending=False)

,feature,impotance
27,col_15,0.084573
1,amount,0.075944
15,col_3,0.064624
8,is_gender_undefined,0.060546
0,bin,0.050987
2,bank_currency,0.046702
3,hour,0.045661
7,latitude,0.037798
18,col_6,0.037417
5,phone_2_norm,0.036518


In [609]:
stat_best.iloc[:,:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
0,3-80-0.2,5.97,10.83,15.34,17.99,18.92,19.62,21.88,24.87,43.06,136.98
4,3-90-0.2,6.05,10.83,14.90,17.26,18.03,20.33,21.14,24.09,40.16,131.32
6,3-90-0.35,3.74,8.27,15.45,17.15,19.35,19.95,19.97,23.23,37.81,125.00
2,3-80-0.35,2.69,7.35,10.91,14.76,16.78,18.54,19.24,22.86,38.33,113.68


In [610]:
data = {'feature': train_df.columns , 'impotance': model.feature_importances_}
df = pd.DataFrame(data)
df.sort_values(by='impotance', ascending=False)

,feature,impotance
1,amount,0.206445
0,bin,0.147512
2,bank_currency,0.142234
3,hour,0.141126
5,phone_2_norm,0.115113
7,latitude,0.109051
6,longitude,0.085222
4,day_of_week,0.053297
